In [38]:
class Knapsack:
  def __init__(self, knapsack, weights, values):
    self.knapsack = knapsack

    self.w = knapsack.dot(weights)
    self.v = knapsack.dot(values)

In [86]:
class GeneticAlg:
  import numpy as np

  def __init__(self, weights, values, max_weight, 
               num_generations=10, population_size=10, mutation_prob=0.15):
    self.weights = weights
    self.values = values
    self.max_weight = max_weight

    self.population_size = population_size
    self.num_generations = num_generations
    self.mutation_prob = mutation_prob

    self.population = self.initialize_population(self.population_size)
    self.best_solution = self.get_best_solution(self.population)

  def run(self):
    for i in range(self.num_generations):
      parents = self.select_parents()
      new_gen = self.create_new_gen(parents)

      best = self.get_best_solution(new_gen)
      if best is not None and best.v > self.best_solution.v:
        self.best_solution = best

      self.population = new_gen

    return self.population



  def select_parents(self):
    parents = []

    values = [x.v + 1 for x in self.population]
    s = sum(values)

    for _ in range(self.population_size):
      parents.append(self.roulette(values, s))

    return parents

  def roulette(self, values, s):
    rng = np.random.default_rng()
    r = rng.integers(s)

    current = 0
    for i, val in enumerate(values):
      current += val
      if current > r:
        return self.population[i]


  def create_new_gen(self, parents):
    new_gen = []

    for p1, p2 in zip(parents[0::2], parents[1::2]):
      new_gen.extend(self.generate_children(p1, p2))

    self.sort_population(new_gen)
    return new_gen

  
  def generate_children(self, p1, p2):
    gen1 = p1.knapsack
    gen2 = p2.knapsack

    gen1, gen2 = self.crossover(gen1, gen2)
    self.mutate(gen1)
    self.mutate(gen2)

    return (Knapsack(gen1, self.weights, self.values), 
            Knapsack(gen2, self.weights, self.values))
    

  def crossover(self, p1, p2):
    rng = np.random.default_rng()
    r = rng.integers(len(p1))

    c1 = p1[:r]
    c1 = np.concatenate([c1, p2[r:]])

    c2 = p1[r:]
    c2 = np.concatenate([c2, p2[:r]])
    
    return c1, c2

  
  def mutate(self, gen):
    rng = np.random.default_rng()
    r = rng.random()

    if r < self.mutation_prob:
      i = rng.integers(len(gen))
      gen[i] = (gen[i] + 1) % 2

    
  def initialize_population(self, size):
    p = []

    while len(p) < size:
      p.append(self.generate_random_solution())

    self.sort_population(p)
    return p

  def generate_random_solution(self):
    rng = np.random.default_rng()
    while True:
      k = rng.integers(1, size=len(self.weights), endpoint=True)

      solution = Knapsack(k, self.weights, self.values)
      
      if self.check_solution(solution):
        return solution

  def check_solution(self, solution):
    if solution.w > self.max_weight:
      return False

    return True

  def sort_population(self, p):
    p.sort(key=lambda x: x.v, reverse=True)

  def get_best_solution(self, population):
    for solution in population:
      if self.check_solution(solution):
        return solution


In [87]:
import numpy as np

In [96]:
w = np.array([63, 21,  2, 32, 13, 80, 19, 37, 56, 41, 14,  8, 32, 42,  7])
v = np.array([13,  2, 20, 10,  7, 14,  7,  2,  2,  4, 16, 17, 17,  3, 21])

max_weight = 275

In [124]:
ga = GeneticAlg(w, v, max_weight, num_generations=25, population_size=50, mutation_prob=0.2)

In [125]:
pop = ga.run()

In [126]:
b = ga.best_solution

In [127]:
b.knapsack

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1])

In [128]:
b.w

272

In [129]:
b.v

137

In [130]:
for p in pop:
  print(p.knapsack, p.w, p.v)

[1 1 1 1 1 1 1 1 0 1 1 1 1 1 1] 411 153
[1 1 1 1 1 1 1 0 1 1 1 1 1 1 1] 430 153
[1 0 1 1 1 1 1 1 1 1 1 1 1 1 1] 446 153
[1 0 1 1 1 1 1 1 1 1 1 1 1 0 1] 404 150
[1 0 1 1 1 1 1 1 1 1 1 1 1 0 1] 404 150
[1 0 1 1 1 1 1 1 1 1 1 1 1 0 1] 404 150
[1 0 1 1 0 1 1 1 1 1 1 1 1 1 1] 433 146
[1 0 1 1 1 1 0 1 1 1 1 1 1 1 1] 427 146
[1 0 1 1 0 1 1 1 1 1 1 1 1 1 1] 433 146
[1 1 1 0 1 1 1 1 1 1 1 1 1 1 1] 435 145
[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1] 383 140
[1 1 1 1 1 1 1 1 1 1 1 1 0 1 1] 435 138
[1 1 1 0 0 1 1 1 1 1 1 1 1 1 1] 422 138
[1 1 1 1 1 1 1 1 1 1 0 1 1 0 1] 411 136
[1 1 0 1 1 1 1 1 1 1 1 1 1 1 1] 465 135
[0 0 1 1 0 1 1 1 1 1 1 1 1 1 1] 370 133
[1 0 0 1 1 1 1 1 1 1 1 1 1 1 1] 444 133
[1 0 1 0 1 1 0 1 1 1 1 1 1 0 1] 353 133
[1 1 1 1 1 1 1 0 1 0 1 0 1 1 1] 381 132
[1 1 1 1 1 0 1 0 1 0 1 1 1 0 1] 267 132
[1 1 1 0 1 0 1 1 1 1 1 1 1 1 1] 355 131
[1 0 1 0 0 1 1 1 0 1 1 1 1 0 1] 303 131
[0 1 1 0 1 1 1 0 1 1 1 1 1 1 1] 335 130
[1 1 1 0 1 1 1 1 1 1 1 0 1 1 1] 427 128
[0 1 1 1 0 1 1 0 0 1 1 1 1 0 1] 256 128
